# InvokeAI in Google Colab
Make sure to enable GPU, Edit > Notebook Settings > Hardware accelerator > GPU <br>
This takes between 10-12 mins to start fully, + about 3 mins per custom model.<br>
To start, Click "Runtime" > "Run All". Alternaivly you can click the "play" button on each step below one after the other, No need to wait for the previous steps to finish as they will join a queue.<br>

##NOTE: As of 2023-07-04, InvokeAI version 3.0.0 is in Beta. Both versions are available here. Please note: Version 3 is unsatble untill full release, at that time this comment will be removed.


In [ ]:
#@title 1. Configuration { display-mode: "form" }
#@markdown #Instance Type
#@markdown Persistent = Stores images in your Google drive, in a folder called "/invokeaiOutput".<br>
#@markdown Temporary = Images are stored in the runtime and are removed when the runtime ends or crashes. Make sure to download your images! <br>
Version = "2.3.5.post2 (Stable)" #@param ['2.3.5.post2 (Stable)','Version 3 (Testing)'] {type:"string"}
Type = "Persistent" #@param ['Persistent','Temporary'] {type:"string"}

#@markdown ---

#@markdown #Xformers
#@markdown Xformers makes this "memory efficient" so larger images (above 1024x1024) can be made, However it can be less stable and may generate blank outputs. It is recomended that you set it to "Enabled" unless you experiance issues with your models.
xformers = "Enabled" #@param ['Enabled' ,'Disabled'] {type:"string"}

#@markdown ---

#@markdown #NSFW Checker
#@markdown This checks outputs for "Not Safe For Work" content and obscures the image. This is not 100% reliable.
nsfw_checker = "Enabled" #@param ['Enabled' ,'Disabled'] {type:"string"}

#@markdown ---

#@markdown #Custom Models
#@markdown Custom Diffuser Model
custom_diffuser_hugging_face_id = "kebab111/crosskemono" #@param ["None","kebab111/crosskemono", "dreamlike-art/dreamlike-photoreal-2.0", "stablediffusionapi/disney-pixal-cartoon"] {allow-input: true}
#@markdown Custom Safetensor or Checkpoint models <br>
custom_model_url = "None" #@param ["None","https://huggingface.co/ffggggfg/Crosskemono/resolve/main/crosskemonoFurryModel_crosskemono25.safetensors"] {allow-input: true}
custom_vae_url = "None" #@param ["None","https://huggingface.co/ffggggfg/Crosskemono/resolve/main/Crosskemono2.vae.pt"] {allow-input: true}

#@markdown If you have selecfted a custom diffuser model will be auto imported and selectable from the dropdown menu in app. <br> <br>
#@markdown Custom .safetensor or .checkpoint models are stored in /content/invokeai/CustomModels and must be manually added via the GUI after it starts. Your Custom VAE will be stored in /content/invokeai/CustomVae <br>

#@markdown ---

#@markdown #NGROK
#@markdown ####Recomended, as it is a more stable connection, but it not required.<br>
#@markdown ####Sign up and get your free token at https://dashboard.ngrok.com/get-started/your-authtoken
#@markdown An NGROK token is needed for the more stable connection, however this project gives 2 options on how to connect, if you are happy with the less stable "Localtunnel" Leave this as "None".<br>
ngrok_token = "None" #@param ['None'] {allow-input: true}

In [ ]:
#@title 2.1. Build and Configure. (V2.3.5.post2) { display-mode: "form" }
#@markdown This takes about 10-12 mins to finish + some extra time to download custom Checkpoint / Safetensor models. { display-mode: "form" }
%%capture
if Version == "2.3.5.post2 (Stable)":
  file_path = '/content/invokeai'

  #Set up temporary storage if running in "Temporary" mode.
  if Type == "Temporary":
      outpath = '/content/invokeai/output'
      import os
      if not os.path.exists(file_path):
        os.makedirs(file_path)

  # Mount and set up Google drive if running in "Persistent" mode.
  if Type == "Persistent":
    outpath = '/content/drive/MyDrive/invokeaiOutput'
    import os
  from google.colab import drive
  if not os.path.exists(file_path):
    drive.mount('/content/drive')
    if not os.path.exists(file_path):
      os.makedirs(file_path)


  #Install Python + Pip
  !sudo apt-get update -y
  !sudo apt-get install python3.9 python3-pip python3.9-distutils python3.9-dev libpython3.9-dev python3.9-venv -y

  #Change alternatives
  !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 3

  #Update pip
  %cd {file_path}
  !curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
  !python3 get-pip.py --force-reinstall
  !python -m pip install --upgrade pip
  !pip install --upgrade setuptools
  !pip install pillow==9.3.0

  #Handle colab dependencies
  !ln -s /usr/local/lib/python3.8/dist-packages/google /usr/local/lib/python3.9/dist-packages/google
  !python3 -m pip install ipython ipython_genutils ipykernel jupyter_console prompt_toolkit httplib2 astor

  #Create InvokeAI root
  import os
  os.environ['INVOKEAI_ROOT'] = file_path
  if not os.path.exists(file_path):
    os.makedirs(invokeai_root)

  #Create the virtual environment
  %cd {file_path}
  !python -m venv .venv --prompt InvokeAI
  !source .venv/bin/activate; python -m pip install --upgrade pip
  !source .venv/bin/activate; pip install InvokeAI[xformers]==2.3.5.post2 --use-pep517 --extra-index-url https://download.pytorch.org/whl/cu117
  !source .venv/bin/activate; python -m pip uninstall torchmetrics==1.0.0 -y
  !source .venv/bin/activate; python -m pip install torchmetrics==0.7.0
  !mkdir {outpath}

  #Edit invokeai.init and download default models
  !cd {file_path}
  !source .venv/bin/activate; invokeai-configure --yes
  !sed -i 's/--max_loaded_models=2/--max_loaded_models=1/' invokeai.init
  if xformers == "Disabled":
    !sed -i 's/--xformers/--no-xformers/' invokeai.init
  if nsfw_checker == "Disabled":
    !sed -i 's/--nsfw_checker/--no-nsfw_checker/' invokeai.init


  #Download custom checkpoint models
  if custom_model_url != "None":
    !mkdir {file_path}/CustomModels
    %cd {file_path}/CustomModels
    !wget -nc {file_path}/CustomModels {custom_model_url}

  #Download custom VAE
  if custom_vae_url != "None":
    !mkdir {file_path}/CustomVae
    %cd {file_path}/CustomVae
    !wget -nc {file_path}/CustomVae {custom_vae_url}

In [ ]:
#@title 2.2. Build and Configure. (V3.0.0.4b) { display-mode: "form" }
#@markdown This takes about 10-15 mins to finish + some extra time to download custom models. <br>
#@markdown Verion 3 auto Imports all model typed Models at runtime.
%%capture
if Version == "Version 3 (Testing)":
  file_path = '/content/invokeai'

  #Set up temporary storage if running in "Temporary" mode.
  if Type == "Temporary":
      outpath = '/content/invokeai/output'
      import os
      if not os.path.exists(file_path):
        os.makedirs(file_path)

  # Mount and set up Google drive if running in "Persistent" mode.
  if Type == "Persistent":
    outpath = '/content/drive/MyDrive/invokeaiOutputV3'
    import os
    from google.colab import drive
    if not os.path.exists(file_path):
      drive.mount('/content/drive')
      if not os.path.exists(file_path):
        os.makedirs(file_path)


  #Install Python + Pip
  !sudo apt-get update -y
  !sudo apt-get install python3.9 python3-pip python3.9-distutils python3.9-dev libpython3.9-dev python3.9-venv -y

  #Change alternatives
  !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 3

  #Update pip
  %cd {file_path}
  !curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
  !python3 get-pip.py --force-reinstall
  !python -m pip install --upgrade pip
  !pip install --upgrade setuptools
  !pip install pillow==9.3.0

  #Handle colab dependencies
  !ln -s /usr/local/lib/python3.8/dist-packages/google /usr/local/lib/python3.9/dist-packages/google
  !python3 -m pip install ipython ipython_genutils ipykernel jupyter_console prompt_toolkit httplib2 astor

  #Create InvokeAI root
  import os
  os.environ['INVOKEAI_ROOT'] = file_path
  if not os.path.exists(file_path):
    os.makedirs(invokeai_root)

  #Create the virtual environment
  %cd {file_path}
  !python -m venv .venv --prompt InvokeAI
  !source .venv/bin/activate; python -m pip install --upgrade pip
  !source .venv/bin/activate; pip install "invokeai @ https://github.com/invoke-ai/InvokeAI/archive/refs/tags/v3.0.0+b4.zip" --use-pep517 --extra-index-url https://download.pytorch.org/whl/cu117
  #Once PIP installs are available, Migrate to the below install option, instead of the above GIT hub pull.
  #!source .venv/bin/activate; pip install InvokeAI[xformers]==VERSION --use-pep517 --extra-index-url https://download.pytorch.org/whl/cu117
  !mkdir {outpath}

  #Edit invokeai.init and download default models
  !cd {file_path}
  !source .venv/bin/activate; invokeai-configure --yes
  !sed -i 's/max_loaded_models: 2/max_loaded_models: 1/' invokeai.yaml
  if xformers == "Disabled":
    !sed -i 's/xformers_enabled: true/xformers_enabled: false/' invokeai.yaml
  if nsfw_checker == "Disabled":
    !sed -i 's/nsfw_checker: true/nsfw_checker: false/' invokeai.yaml


  #Download custom checkpoint models
  if custom_model_url != "None":
    #!mkdir {file_path}/CustomModels
    #%cd {file_path}/CustomModels
    #!wget -nc {file_path}/CustomModels {custom_model_url}
    !mkdir {file_path}/CustomModels
    %cd {file_path}/CustomModels
    !wget -nc {file_path}/autoimport/main {custom_model_url}


  #Download custom VAE
  if custom_vae_url != "None":
    #!mkdir {file_path}/CustomVae
    #%cd {file_path}/CustomVae
    #!wget -nc {file_path}/CustomVae {custom_vae_url}
    %cd {file_path}/autoimport/vae
    !wget -nc {file_path}/CustomVae {custom_vae_url}

  #Import Diffuser Model
  if custom_diffuser_hugging_face_id != "None":
    %cd {file_path}/autoimport/main
    !git clone https://huggingface.co/{custom_diffuser_hugging_face_id}

  %cd {file_path}

In [ ]:
#@title 3. Connect via NGROK - Recomended. { display-mode: "form" }
#@markdown This link will start working when the next step tells you to connect to localhost.<br>
#@markdown If you are not using NGROK or if NGROK fails to start, See the next step for an alternate connection method.<br>
#@markdown You can ignore this step if you have not suplied a NGROK token in the configuration section.

if ngrok_token != "None":
  %cd {file_path}
  !pip install pyngrok --quiet
  from pyngrok import ngrok
  ngrok.kill()
  ngrok.set_auth_token(ngrok_token)
  public_url = ngrok.connect(9090).public_url
  print(f'InvokeAI Public URL: {public_url}')

if ngrok_token == "None":
  print("You have not suppied an NGROK token.")
  print("Please follow the connection process for localtunnel below.")

In [ ]:
#@title 4. Start InvokeAI + "localtunnel" { display-mode: "form" }
#@markdown ### Starting the App
#@markdown This step takes either 2-3 mins to start InvokeAI or 5-6 mins if you have selected a diffuser model. <br>
#@markdown Diffuser models that are auto imported are NOT selected by default and must be chosen from the dropdown menu in the top right.

#@markdown ---

#@markdown ### How to connect to localtnnel
#@markdown A "localtunnel" Interface connection is generated here, To use this, please do the following: <br>
#@markdown 1) Copy the IP address appears in line 2 or 3 of this cells output. <br>
#@markdown 2) Click the random "https://XXX-YYY-ZZZ.loca.lt" link that is generated in (Aproximatly) like 8. <br>
#@markdown 3) Paste the IP into the provided box and submit. <br>
#@markdown Note: An error of "502 Bad Gateway" typically is an error at Localtunnels end.<br><br>
if Version == "2.3.5.post2 (Stable)":
  %cd {file_path}
  print("Localtunnel IP address:")
  !curl ipv4.icanhazip.com
  !npm install -g localtunnel
  if custom_diffuser_hugging_face_id != "None":
    !npx localtunnel --port 9090 & !source .venv/bin/activate; invokeai --web --host 0.0.0.0 --outdir={outpath} --autoimport {custom_diffuser_hugging_face_id}
  if custom_diffuser_hugging_face_id == "None":
    !npx localtunnel --port 9090 & !source .venv/bin/activate; invokeai --web --host 0.0.0.0 --outdir={outpath}


if Version == "Version 3 (Testing)":
  %cd {file_path}
  print("Localtunnel IP address:")
  !curl ipv4.icanhazip.com
  !npm install -g localtunnel
  !npx localtunnel --port 9090 & !source .venv/bin/activate; invokeai-web --outdir={outpath}


In [ ]:
#@title 5. Temporary Mode - Zip Output for Download. { display-mode: "form" }
#@markdown Google Colab does not give a nice way to download multiple files at once.<br>
#@markdown When running in Temporary Mode, This will compress all images into a single "output.zip" to make downloading everything simple!<br>
#@markdown This file can be downloaded using the Colab file browser: /invokeai/output/Output.zip <br>
#@markdown For this step to run you must manually stop step 4.

if Type == "Temporary":
  !mkdir /content/zip
  !rm {outpath}/Output.zip
  !rm -r /content/zip/*
  !cp -p {outpath}/*png /content/zip
  !cd /content/zip/
  !zip {outpath}/Output.zip /content/zip/*.png